In [1]:
!pip install selectolax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 12.0 MB/s eta 0:00:00


In [2]:
import requests
import json
from selectolax.parser import HTMLParser
import csv
import requests
import json
import pandas as pd

In [3]:
# response.encoding = 'utf-8'
# html = response.text
# tree = HTMLParser(html)
# script = tree.css_first('script[id="initial_state_script"]').text()
# script = script[23:-1]
# data = json.loads(script)
# offers = data['map']['offers']['points']
# # offer_short = data['map']['offers']['points'][0]
# print(offers)

In [4]:
###NEW
def get_offers(offers):
    # Prepare a list for storing parsed data
    parsed_offers = []

    for offer in offers:
        parsed_data = {
            'url_offer': offer.get('url'),
            'rooms_total': offer.get('roomsTotal'),
            'offer_type': offer.get('offerType'),
            'offer_id': offer.get('offerId'),
            'update_date': offer.get('updateDate'),
            'flat_type': offer.get('flatType'),
            'kitchen_space': offer.get('kitchenSpace', {}).get('value'),
            'ceiling_height': offer.get('ceilingHeight'),

            # Parsing predicted_price
            'predicted_price_min': None,
            'predicted_price_max': None,
            'predicted_price_value': None,

            # Parsing price
            'price': offer.get('price', {}).get('value'),
            'price_per_meter': offer.get('price', {}).get('pricePerPart',{}).get('value'),

            # Parsing location details
            'address': offer.get('location', {}).get('address'),
            'latitude': offer.get('location', {}).get('point', {}).get('latitude'),
            'longitude': offer.get('location', {}).get('point', {}).get('longitude'),

            # Parsing metro details
            'metro_name': None,
            'time_to_metro': None,

            # Parsing airport details
            'closest_airport': None,
            'time_to_airport': None,

            # Parsing other details
            'mortgage_possible': offer.get('transactionConditionsMap', {}).get('MORTGAGE'),
            'renovation': offer.get('apartment', {}).get('renovation'),
            'floor': offer.get('floorsOffered'),
            'total_floors': offer.get('floorsTotal'),
            'year_built': offer.get('building', {}).get('builtYear'),
            'material': offer.get('building', {}).get('buildingType'),
            'parking': offer.get('building', {}).get('parkingType'),
            'balcony': offer.get('house', {}).get('balconyType'),
            'heating': offer.get('building',{}).get('heatingType'),
            'is_apart': offer.get('house',{}).get('apartments'),
            #'condition': offer.get('condition'),
            #'furniture': offer.get('furniture'),
            #'proximity_to_services': offer.get('proximityToServices'),
            'area': offer.get('area', {}).get('value')

        }

        # If these keys exist in the offer, parse them
        if 'predictions' in offer and 'predictedPrice' in offer['predictions']:
            predicted_price = offer['predictions']['predictedPrice']
            parsed_data['predicted_price_min'] = predicted_price.get('min')
            parsed_data['predicted_price_max'] = predicted_price.get('max')
            parsed_data['predicted_price_value'] = predicted_price.get('value')

        if 'location' in offer:
            if 'metroList' in offer['location']:
                metro_list = offer['location']['metroList']
                nearest_metro = min(metro_list, key=lambda x: x.get('timeToMetro', float('inf')))
                parsed_data['metro_name'] = nearest_metro.get('name')
                parsed_data['time_to_metro'] = nearest_metro.get('timeToMetro')

            if 'airports' in offer['location']:
                airports = offer['location']['airports']
                closest_airport = airports[0]
                parsed_data['closest_airport'] = closest_airport.get('name')
                parsed_data['time_to_airport'] = closest_airport.get('timeOnCar')

        parsed_offers.append(parsed_data)

    return parsed_offers


In [5]:
url = "https://realty.ya.ru/sankt-peterburg/kupit/kvartira/"

# Prepare a list for storing all parsed data
all_parsed_offers = []

# Iterate through all pages
page = 1
while page < 25:
    params = (('page', page),)
    response = requests.get(url, params=params)

    print(f"Processing page: {response.url}")

    if response.status_code == 200:
        response.encoding = 'utf-8'
        html = response.text
        tree = HTMLParser(html)
        script = tree.css_first('script[id="initial_state_script"]').text()
        script = script[23:-1]
        data = json.loads(script)
        offers = data['map']['offers']['points']

        parsed_offers = get_offers(offers)

        # Save the parsed offers into all_parsed_offers list
        all_parsed_offers.extend(parsed_offers)

        page += 1
    else:
      print('error')

# Specify the fieldnames for the CSV
fieldnames = [
  'url_offer',
  'rooms_total',
  'offer_type',
  'offer_id',
  'update_date',
  'flat_type',
  'kitchen_space',
  'ceiling_height',
  'predicted_price_min',
  'predicted_price_max',
  'predicted_price_value',
  'price',
  'price_per_meter',
  'address',
  'latitude',
  'longitude',
  'metro_name',
  'time_to_metro',
  'closest_airport',
  'time_to_airport',
  'mortgage_possible',
  'renovation',
  'floor',
  'total_floors',
  'year_built',
  'material',
  'parking',
  'balcony',
  'heating',
  'is_apart',
  'area',
  'infrastructure_description',
  'infrastructure_level',
  'infrastructure_max_level',
  'price-rent_description',
  'price-rent_level',
  'price-rent_max-level',
  'profitability_description',
  'profitability_level',
  'profitability_max-level',
  'transport_description',
  'transport_level',
  'transport_max-level',
  'carsharing_description',
  'carsharing_level',
  'carsharing_max-level']

# Write data to CSV file
with open('offers.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for offer in all_parsed_offers:
        print(offer)
        writer.writerow(offer)
    print('количество оферов: ', len(all_parsed_offers))

Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=1
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=2
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=3
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=4
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=5
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=6
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=7
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=8
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=9
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=10
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=11
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?page=12
Processing page: https://realty.ya.ru/sankt-peterburg/kupit/kvartira/?pag

In [8]:
offers[0]

{'unsignedInternalUrl': '//realty.ya.ru/offer/7390093081101672768',
 'appMiddleSnippetImages': ['//avatars.mds.yandex.net/get-realty/197452/offer.5126982595923256929.4596328517667187313/app_snippet_middle',
  '//avatars.mds.yandex.net/get-realty/193295/offer.7390093081101672768.4009298615748269548/app_snippet_middle',
  '//avatars.mds.yandex.net/get-realty/8075677/offer.2698151376781924754.2598485003381989660/app_snippet_middle',
  '//avatars.mds.yandex.net/get-realty/4478263/offer.2913478258422217537.2598485003381066139/app_snippet_middle',
  '//avatars.mds.yandex.net/get-realty/8075677/offer.6609906126570601482.2598485003380142618/app_snippet_middle',
  '//avatars.mds.yandex.net/get-realty/8283496/offer.7150797836198135985.2598485003379219097/app_snippet_middle',
  '//avatars.mds.yandex.net/get-realty/9742424/offer.1826945957265250013.2598485003378295576/app_snippet_middle',
  '//avatars.mds.yandex.net/get-realty/9742424/offer.7907743921277554193.2598485003377372055/app_snippet_middl

In [12]:
# try:
#     data = json.loads(offers[0]) # Загрузка JSON из строки
#     pretty_json = json.dumps(data, indent=4) # Переформатирование с отступом в 4 пробела
# except json.JSONDecodeError as e:
#     print(f"Invalid JSON: {e}")


for offer in offers[0]:

  #print(offer['price'])

  data = offer # уже готовый словарь

  pretty_json = json.dumps(data, indent=4)
  print(pretty_json)

"unsignedInternalUrl"
"appMiddleSnippetImages"
"promoted"
"building"
"fullImages"
"allowedCommunicationChannels"
"price"
"appMiddleImages"
"origImages"
"commissioningDateIndexValue"
"hasPaidCalls"
"exclusive"
"floorsTotal"
"area"
"partnerInternalId"
"floorsOffered"
"author"
"minicardImages"
"active"
"vas"
"mainImages"
"paidReportAccessibility"
"livingSpace"
"suspicious"
"dealStatus"
"creationDate"
"relevance"
"tags"
"offerLinks"
"appLargeSnippetImages"
"large1242Images"
"trustedOfferInfo"
"offerId"
"yandexRent"
"shareUrl"
"freeReportAccessibility"
"withExcerpt"
"vasAvailable"
"flatType"
"placement"
"socialPessimization"
"clusterHeader"
"trust"
"updateDate"
"internal"
"appSmallSnippetImages"
"appLargeImages"
"obsolete"
"description"
"clusterId"
"house"
"appMiniSnippetImages"
"platform"
"notForAgents"
"uid"
"offerType"
"chargeForCallsType"
"premium"
"transactionConditionsMap"
"cashbackYandexPlusPercent"
"newBuilding"
"cosmicImages"
"callCenter"
"partnerName"
"extImages"
"remoteReview"
"s

In [ ]:
# with open('data.json', 'w') as f:
#     json.dump(data, f)

In [6]:
df = pd.read_csv('offers.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   url_offer              600 non-null    object 
 1   rooms_total            524 non-null    float64
 2   offer_type             600 non-null    object 
 3   offer_id               600 non-null    int64  
 4   update_date            575 non-null    object 
 5   flat_type              600 non-null    object 
 6   kitchen_space          512 non-null    float64
 7   ceiling_height         462 non-null    float64
 8   predicted_price_min    490 non-null    float64
 9   predicted_price_max    490 non-null    float64
 10  predicted_price_value  490 non-null    float64
 11  price                  600 non-null    int64  
 12  price_per_meter        600 non-null    int64  
 13  address                600 non-null    object 
 14  latitude               600 non-null    float64
 15  longit

In [ ]:
for col in dф.columns:
    print(f"{col}:\n{df[col].unique()}\n")

Output hidden; open in https://colab.research.google.com to view.